In [3]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='raw.csv')

c:\Users\SamOz.py\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-632f9963ed6fd4e7


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 499.80it/s]


Dataset csv downloaded and prepared to C:\Users\SamOz.py\.cache\huggingface\datasets\csv\default-632f9963ed6fd4e7\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 10.64it/s]


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['snippet', 'language'],
        num_rows: 5241868
    })
})

In [17]:
from transformers import AutoTokenizer
model_checkpoint = 'huggingface/CodeBERTa-small-v1'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading: 100%|██████████| 19.0/19.0 [00:00<00:00, 19.0kB/s]
Downloading: 100%|██████████| 480/480 [00:00<00:00, 240kB/s]
Downloading: 100%|██████████| 971k/971k [00:00<00:00, 1.18MB/s]
Downloading: 100%|██████████| 471k/471k [00:05<00:00, 81.4kB/s] 


In [5]:
dataset['train']['snippet'][0]

'type Signature struct {\r\n\tInputs, Outputs map[string]TensorInfo\r\n\tMethodName      string\r\n}\r\n\r\n'

In [9]:
dataset_split = dataset['train'].train_test_split(test_size=.1,train_size=.9)

In [13]:
dataset_split = dataset_split.rename_column('snippet', 'text')

In [15]:
dataset_split = dataset_split.remove_columns('language')

In [16]:
dataset_split

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4717681
    })
    test: Dataset({
        features: ['text'],
        num_rows: 524187
    })
})

In [18]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset_split.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

Parameter 'function'=<function tokenize_function at 0x000001F2002B7168> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 525/525 [00:34<00:00, 15.09ba/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 4717681
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 524187
    })
})

In [19]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 4717681
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 524187
    })
})

In [20]:
chunk_size = 128 
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [21]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

100%|██████████| 525/525 [03:23<00:00,  2.58ba/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2098514
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 231680
    })
})

In [22]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'58: 0x5FC9,\r\n\t2659: 0x5FCB,\r\n\t2660: 0x5FD0,\r\n\t2661: 0x5FD1,\r\n</s><s>\t\texpectedError        string\r\n\t}{\r\n\t\t{\r\n\t\t\tserviceNamesWriter:   func(serviceName string) error { return nil },\r\n\t\t\toperationNamesWriter: func(operation dbmodel.Operation) error { return nil },\r\n</s><s>\tIFT_RADIOMAC                      = 0xbc\r\n\tIFT_RADSL                         = 0x5f\r\n\tIFT_REACHDSL                     '

In [23]:
lm_datasets.save_to_disk('lm_datasets.hf')